<a href="https://colab.research.google.com/github/Mike030668/MIPT_magistratura/blob/main/NLP_generation%20/Project_retrivial_bot/Context_talk_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/Mike030668/MIPT_magistratura.git -q

In [ ]:
import gdown  # библиотека по работе с файлами в том числе и с гугл_диска
import pandas as pd
import numpy as np
import random
import torch
import gc
from transformers import AutoTokenizer, AutoModel
import sys

import warnings
warnings.filterwarnings('ignore')

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MAIN_DIR =  "/content/MIPT_magistratura/NLP_generation/Project_retrivial_bot"
MAX_LENGTH = 128

sys.path.append(MAIN_DIR)

# Load database

In [ ]:
from utils.utils import get_replies, load_weights, CrossEncoderBert

path_df = MAIN_DIR + "/data/talks_dataset.df"
talks_df = pd.read_pickle(path_df)

all_replies = get_replies(talks_df)

# Load model

In [ ]:
url = 'https://drive.google.com/file/d/1yj0VuOJmpwio0h9cLuBs50oG_YzVr86a/view?usp=sharing'
weights =  '/model.pt'
load_weights(url_weights = url, name_model = "cross_encoder", name_weights = weights, main_dir = MAIN_DIR)

cross_model = torch.load(MAIN_DIR +"/models/cross_encoder/model.pt")
cross_model.to(DEVICE)

Downloading...
From (original): https://drive.google.com/uc?id=1yj0VuOJmpwio0h9cLuBs50oG_YzVr86a
From (redirected): https://drive.google.com/uc?id=1yj0VuOJmpwio0h9cLuBs50oG_YzVr86a&confirm=t&uuid=eec11bbf-1f68-45dd-afb9-44c994e57ba2
To: /content/MIPT_magistratura/NLP_generation/Project_retrivial_bot/models/cross_encoder/model.pt
100%|██████████| 266M/266M [00:01<00:00, 143MB/s]


CrossEncoderBert(
  (bert_model): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
            (lin1): 

# Вопрос ответ

In [ ]:
from utils.talk_context import flush_memory, get_best_rand_reply

In [ ]:
question = "What is the live?"  #
print(f"Реплика: {question}")

best_answer, conext_memory,  best_score = get_best_rand_reply(
    cross_model,
    query = question,
    context = "",
    corpus = all_replies,
    max_length=MAX_LENGTH,
    device = DEVICE
    )

print(f"Лучший ответ: {best_answer}\nscore {best_score}")


Реплика: What is the live?
Лучший ответ: But sometimes, after the Siamese twins are joined together, one of the twins dies before birth. The living baby is born with the dead baby still attached.  Sometimes, this dead twin is inside the living person, so even you could have a dead twin inside you and not even know it! 
score 0.9702920913696289


# Иммитация беседы с контекстом

In [ ]:
con = 0
question = input('Реплика: ')
while question != 'stop':
  #print(f"Реплика: {question}")
  if not con: conext_memory = ""
  best_answer, conext_memory, best_score = get_best_rand_reply(
                              cross_model,
                              query = question,
                              context = conext_memory,
                              corpus = all_replies,
                              max_length=MAX_LENGTH,
                              device = DEVICE
                              )
  print(f"Ответ: {best_answer}\n")
  con+=1
  question = input('Реплика: ')

Реплика: Hello
Ответ: Little Bunny Foo-Foo hoppin' through the forestScoopin' up the field mice and boppin' 'em on the headDown came a white angel and she said"Little Bunny Foo-Foo, I don't wanna see youScoopin' up the field mice and boppin' 'em on the head"So now I'm gonna turn you into a worm, mbuh, mBunny Foo-Foo... Eh, beh-POOFLittle Wormy Foo-Foo crawlin' through the forestGettin scooped up by the field mice who mah-m then they bopped 'im on the head

Реплика: How are you?
Ответ: Stan, nunh tunh tunh tunh tunh tunh tunh tunh tunh tunh tunh tunh hunh. Tunh tunh tunh tunh tinh teenh?

Реплика: What do think about rest on the beach?
Ответ: Kyle! Kyle! Get the... Get the- Kyle! Kyle, dude, that was- that was in the balls! Dude, serious- seriously! That was in the balls! No hitting- No hitting in the balls!  Kyle!  Quit it, Kyle. Give it up, Kyle!  Okay... Okay! Okay okay!  You win! You win. I give up.  Hahaha! I had my fingers crossed. 

Реплика: Let's go together?
Ответ: Timmih! Timm